### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import pandas as pd
import numpy as np

# File to Load (Remember to Change These)
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas DataFrames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset.  
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])

## District Summary

* Calculate the total number of schools

* Calculate the total number of students

* Calculate the total budget

* Calculate the average math score 

* Calculate the average reading score

* Calculate the percentage of students with a passing math score (70 or greater)

* Calculate the percentage of students with a passing reading score (70 or greater)

* Calculate the percentage of students who passed math **and** reading (% Overall Passing)

* Create a dataframe to hold the above results

* Optional: give the displayed data cleaner formatting

In [2]:
#Finding the number of different schools in the list
school_list = school_data_complete["school_name"].unique()

#Counting the number of schools
school_count = 0
for school in school_list:
    school_count += 1

#Counting the number of students
total_students = school_data_complete["student_name"].count()

#Summing up the total budgets of the schools (Assumes no two schools have the same budget.)
total_budget = school_data_complete["budget"].unique().sum()
total_budget = "${:,.0f}".format(total_budget)
#Computing and storing the mean reading and math scores
avg_math_score = school_data_complete["math_score"].mean()
avg_read_score = school_data_complete["reading_score"].mean()

#Creating new df with only passing subsets included
both_pass_df = school_data_complete.loc[(school_data_complete["reading_score"]>=70) & (school_data_complete["math_score"]>=70),["student_name"]]
read_only_df = school_data_complete.loc[school_data_complete["reading_score"]>=70, ["student_name"]]
math_only_df = school_data_complete.loc[school_data_complete["math_score"]>=70, ["student_name"]]

#Counting passing students
both_count =  both_pass_df.count()
read_only = read_only_df.count()
math_only = math_only_df.count()

#Computing the percentage of students who passed for the district
percent_both = int(both_count)/total_students*100
percent_read = int(read_only)/total_students*100
percent_math = int(math_only)/total_students*100

#Formatting Total Students
total_students = "{:,}".format(total_students)

#Creating district summary data frame.
district_summary = pd.DataFrame(
    {"Total Schools": [school_count],
    "Total Students": [total_students],
    "Total Budget": [total_budget],
    "Average Math Score": [avg_math_score],
    "Average Reading Score": [avg_read_score],
     "% Passing Math": [percent_math],
     "% Passing Reading": [percent_read],
     "% Overall Passing": [percent_both]
    }
)

#Showing the district summary in the output
district_summary.style.hide_index()


Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
15,"39,170","$24,649,428",78.985371,81.877840,74.980853,85.805463,65.172326


## School Summary

* Create an overview table that summarizes key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * % Overall Passing (The percentage of students that passed math **and** reading.)
  
* Create a dataframe to hold the above results

In [3]:
#Using groupby to determine mean budget and scores
grouped_school = school_data_complete.groupby(["school_name"])
school_summary_df = grouped_school.mean()
school_summary_df.drop('Student ID', inplace=True, axis=1)
school_summary_df.drop("School ID", inplace = True, axis =1)

#Currency format function
def currency(x):
    return "${:,.0f}".format(x)

def integer(x):
    return int(x)

#Creating empty lists
school_type = []
per_student_budget = []
school_per_math = []
school_per_read = []
school_per_over = []

#Looping through subset school data frames and appending data for each school to the correct list.
for school in school_list:
    school_df = school_data_complete[school_data_complete["school_name"]==str(school)]
    school_type_list = school_df["type"].values.tolist()
    school_type.append(school_type_list[0])
    school_budget_list = school_df["budget"].values.tolist()
    per_student_item = float(school_budget_list[0])/school_df["student_name"].count()
    per_student_budget.append(per_student_item)

    #Creating new df for each school with only passing subsets included
    both_pass_df = school_df.loc[(school_df["reading_score"]>=70) & (school_df["math_score"]>=70),["student_name"]]
    read_only_df = school_df.loc[school_df["reading_score"]>=70, ["student_name"]]
    math_only_df = school_df.loc[school_df["math_score"]>=70, ["student_name"]]

    #Counting passing students
    both_count =  both_pass_df.count()
    read_only = read_only_df.count()
    math_only = math_only_df.count()

    #Computing passing percentages and appending them to the correct list
    school_per_over.append(int(both_count)/school_df["student_name"].count()*100)
    school_per_read.append(int(read_only)/school_df["student_name"].count()*100)
    school_per_math.append(int(math_only)/school_df["student_name"].count()*100)

#Adding new columns to dataframe
school_summary_df2 = pd.DataFrame(
    {"school_name": school_list,
    "School Type": school_type,
    "Per Student Budget": per_student_budget,
     "% Passing Math": school_per_math,
     "% Passing Reading": school_per_read,
     "% Overall Passing": school_per_over})

school_summary_df.rename(columns = {'reading_score':'Average Reading Score', 
                                    'math_score':"Average Math Score",
                                    "size":"Total Students", 
                                    "budget":"Average Budget",
                                    }, 
                         inplace = True)

#Merging two dataframes together
school_summary_merge = pd.merge(school_summary_df, school_summary_df2, on = "school_name", how = "outer")

#Reorganizing Columns
school_summary_merge = school_summary_merge[['school_name','School Type','Total Students', 'Average Budget','Per Student Budget','Average Math Score', 'Average Reading Score','% Passing Math', '% Passing Reading', '% Overall Passing']]
school_summary_merge.rename(columns = {"school_name": "School Name"}, inplace =True)

clean_school_summary = school_summary_merge.copy()

#Formatting budget and per student budget columns as currency
school_summary_merge["Average Budget"] = school_summary_merge["Average Budget"].apply(currency)
school_summary_merge["Per Student Budget"] = school_summary_merge["Per Student Budget"].apply(currency)
school_summary_merge["Total Students"] = school_summary_merge["Total Students"].apply(integer)

#Sorting the dataframe alphabetically by school name
school_summary_merge.sort_values(by=["School Name"])


,School Name,School Type,Total Students,Average Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
0,Bailey High School,District,4976,"$3,124,928",$628,77.048432,81.033963,66.680064,81.933280,54.642283
1,Cabrera High School,Charter,1858,"$1,081,356",$582,83.061895,83.975780,94.133477,97.039828,91.334769
2,Figueroa High School,District,2949,"$1,884,411",$639,76.711767,81.158020,65.988471,80.739234,53.204476
3,Ford High School,District,2739,"$1,763,916",$644,77.102592,80.746258,68.309602,79.299014,54.289887
4,Griffin High School,Charter,1468,"$917,500",$625,83.351499,83.816757,93.392371,97.138965,90.599455
5,Hernandez High School,District,4635,"$3,022,020",$652,77.289752,80.934412,66.752967,80.862999,53.527508
6,Holden High School,Charter,427,"$248,087",$581,83.803279,83.814988,92.505855,96.252927,89.227166
7,Huang High School,District,2917,"$1,910,635",$655,76.629414,81.182722,65.683922,81.316421,53.513884
8,Johnson High School,District,4761,"$3,094,650",$650,77.072464,80.966394,66.057551,81.222432,53.539172
9,Pena High School,Charter,962,"$585,858",$609,83.839917,84.044699,94.594595,95.945946,90.540541


## Top Performing Schools (By % Overall Passing)

* Sort and display the top five performing schools by % overall passing.

In [4]:
#Sorting the dataframe by overall pass rate
school_summary_merge.sort_values(by=["% Overall Passing"], ascending=False).head().style.hide_index()

School Name,School Type,Total Students,Average Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Cabrera High School,Charter,1858,"$1,081,356",$582,83.061895,83.975780,94.133477,97.039828,91.334769
Thomas High School,Charter,1635,"$1,043,130",$638,83.418349,83.848930,93.272171,97.308869,90.948012
Griffin High School,Charter,1468,"$917,500",$625,83.351499,83.816757,93.392371,97.138965,90.599455
Wilson High School,Charter,2283,"$1,319,574",$578,83.274201,83.989488,93.867718,96.539641,90.582567
Pena High School,Charter,962,"$585,858",$609,83.839917,84.044699,94.594595,95.945946,90.540541


## Bottom Performing Schools (By % Overall Passing)

* Sort and display the five worst-performing schools by % overall passing.

In [5]:
#Sorting the dataframe by overall pass rate
school_summary_merge.sort_values(by=["% Overall Passing"]).head().style.hide_index()

School Name,School Type,Total Students,Average Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Rodriguez High School,District,3999,"$2,547,363",$637,76.842711,80.744686,66.366592,80.220055,52.988247
Figueroa High School,District,2949,"$1,884,411",$639,76.711767,81.158020,65.988471,80.739234,53.204476
Huang High School,District,2917,"$1,910,635",$655,76.629414,81.182722,65.683922,81.316421,53.513884
Hernandez High School,District,4635,"$3,022,020",$652,77.289752,80.934412,66.752967,80.862999,53.527508
Johnson High School,District,4761,"$3,094,650",$650,77.072464,80.966394,66.057551,81.222432,53.539172


## Math Scores by Grade

* Create a table that lists the average Reading Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

  * Create a pandas series for each grade. Hint: use a conditional statement.
  
  * Group each series by school
  
  * Combine the series into a dataframe
  
  * Optional: give the displayed data cleaner formatting

In [6]:
#I've done this two ways: once with a nested groupby statement and once with a for loop

#Creating a math_summary df using a nested groupby statement
#Copying and renaming columns in a df for analysis
math_df = school_data_complete.copy()
math_df = math_df[["school_name", "grade", "math_score"]]
math_df.rename(columns = {'math_score':"Average Math Score",
                                      'grade': "Grade",
                                      'school_name':"School Name"}
                            , inplace = True)

#Using groupby to determine mean math scores by grade
grouped_grade = math_df.groupby(["School Name","Grade"])
math_summary_df = grouped_grade.mean()
print(math_summary_df)

#Creating a math summary df using a for loop
#Copying the original dataframe
school_df2 = school_data_complete.copy()

#Creating empty lists for the math averages
frosh_list = []
soph_list = []
junior_list = []
senior_list = []

#Looping through dataframe by schools and grouping students by grade level
for school in school_list:
    frosh_df = school_df2[(school_df2["grade"]=="9th") & (school_df2["school_name"]==str(school))]
    frosh_list.append(frosh_df["math_score"].mean())
    soph_df = school_df2[(school_df2["grade"]=="10th") & (school_df2["school_name"]==str(school))]
    soph_list.append(soph_df["math_score"].mean())                        
    junior_df = school_df2[(school_df2["grade"]=="11th") & (school_df2["school_name"]==str(school))]
    junior_list.append(junior_df["math_score"].mean())
    senior_df = school_df2[(school_df2["grade"]=="12th") & (school_df2["school_name"]==str(school))]
    senior_list.append(senior_df["math_score"].mean())

#Creating a math summary frame using a dictionary of the grade level lists.
math_summary_df2 = pd.DataFrame(
    {"School Name": school_list,
    "9th": frosh_list,
    "10th": soph_list,
    "11th": junior_list,
    "12th": senior_list})

#Sorting the dataframe alphabetically by school name
math_summary_df2.sort_values(by=["School Name"]).style.hide_index()


                             Average Math Score
School Name           Grade                    
Bailey High School    10th            76.996772
                      11th            77.515588
                      12th            76.492218
                      9th             77.083676
Cabrera High School   10th            83.154506
                      11th            82.765560
                      12th            83.277487
                      9th             83.094697
Figueroa High School  10th            76.539974
                      11th            76.884344
                      12th            77.151369
                      9th             76.403037
Ford High School      10th            77.672316
                      11th            76.918058
                      12th            76.179963
                      9th             77.361345
Griffin High School   10th            84.229064
                      11th            83.842105
                      12th            83

School Name,9th,10th,11th,12th
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164
Hernandez High School,77.438495,77.337408,77.136029,77.186567
Holden High School,83.787402,83.429825,85.000000,82.855422
Huang High School,77.027251,75.908735,76.446602,77.225641
Johnson High School,77.187857,76.691117,77.491653,76.863248
Pena High School,83.625455,83.372000,84.328125,84.121547


## Reading Score by Grade 

* Perform the same operations as above for reading scores

In [7]:
#I've done this two ways, once with a nested groupby and once with a for loop

#Creating a reading summary df using a nested groupby statement
#Copying and renaming columns in a df for analysis
read_df = school_data_complete.copy()
read_df = read_df[["school_name", "grade", "reading_score"]]
read_df.rename(columns = {'reading_score':"Average Reading Score",
                                      'grade': "Grade",
                                      'school_name':"School Name"}
                            , inplace = True)

#Using groupby to determine mean reading scores by grade
grouped_grade = read_df.groupby(["School Name","Grade"])
read_summary_df = grouped_grade.mean()
print(read_summary_df)

#Using the for loop method to create a reading summary df
#Creating empty lists for the reading averages
frosh_list = []
soph_list = []
junior_list = []
senior_list = []

#Looping through dataframe by schools and grouping students by grade level
for school in school_list:
    frosh_df = school_df2[(school_df2["grade"]=="9th") & (school_df2["school_name"]==str(school))]
    frosh_list.append(frosh_df["reading_score"].mean())
    soph_df = school_df2[(school_df2["grade"]=="10th") & (school_df2["school_name"]==str(school))]
    soph_list.append(soph_df["reading_score"].mean())                        
    junior_df = school_df2[(school_df2["grade"]=="11th") & (school_df2["school_name"]==str(school))]
    junior_list.append(junior_df["reading_score"].mean())
    senior_df = school_df2[(school_df2["grade"]=="12th") & (school_df2["school_name"]==str(school))]
    senior_list.append(senior_df["reading_score"].mean())

#Creating a reading summary frame using a dictionary of the grade level lists.
read_summary_df2 = pd.DataFrame(
    {"School Name": school_list,
    "9th": frosh_list,
    "10th": soph_list,
    "11th": junior_list,
    "12th": senior_list})

#Sorting the dataframe alphabetically by school name
read_summary_df2.sort_values(by=["School Name"]).style.hide_index()                              


                             Average Reading Score
School Name           Grade                       
Bailey High School    10th               80.907183
                      11th               80.945643
                      12th               80.912451
                      9th                81.303155
Cabrera High School   10th               84.253219
                      11th               83.788382
                      12th               84.287958
                      9th                83.676136
Figueroa High School  10th               81.408912
                      11th               80.640339
                      12th               81.384863
                      9th                81.198598
Ford High School      10th               81.262712
                      11th               80.403642
                      12th               80.662338
                      9th                80.632653
Griffin High School   10th               83.706897
                      11th     

School Name,9th,10th,11th,12th
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699
Hernandez High School,80.866860,80.660147,81.396140,80.857143
Holden High School,83.677165,83.324561,83.815534,84.698795
Huang High School,81.290284,81.512386,81.417476,80.305983
Johnson High School,81.260714,80.773431,80.616027,81.227564
Pena High School,83.807273,83.612000,84.335938,84.591160


## Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

In [8]:
#Copying the unformatted school summary for use here.
spending_df = clean_school_summary.copy()
spending_df.drop(["Total Students"], inplace = True, axis = 1)
spending_df.drop(["Average Budget"], inplace = True, axis = 1)

#Bucket List broken down by per pupil spending
bucket_list = [0, 584.9, 629.9, 644.9, 700]
bucket_list_names = ["<=$585", "$586-630", "$631-645", "$646-680"]
spending_df["PPE Range"] = pd.cut(spending_df["Per Student Budget"], 
                                           bucket_list, labels = bucket_list_names, include_lowest=True)
#Creating a groupby for that set of bins
spending_summary_df = spending_df.groupby("PPE Range").mean()

spending_summary_df.drop(["Per Student Budget"], inplace = True, axis = 1)

#Formatting output
def finalformat(x):
    return "{:.2f}".format(x)

def percentformat(x):
    return "{:.2f}%".format(x)

spending_summary_df["Average Math Score"] = spending_summary_df["Average Math Score"].apply(finalformat)
spending_summary_df["Average Reading Score"] = spending_summary_df["Average Reading Score"].apply(finalformat)
spending_summary_df["% Passing Math"] = spending_summary_df["% Passing Math"].apply(percentformat)
spending_summary_df["% Passing Reading"] = spending_summary_df["% Passing Reading"].apply(percentformat)
spending_summary_df["% Overall Passing"] = spending_summary_df["% Overall Passing"].apply(percentformat)

spending_summary_df

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
PPE Range,,,,,
<=$585,83.46,83.93,93.46%,96.61%,90.37%
$586-630,81.90,83.16,87.13%,92.72%,81.42%
$631-645,78.52,81.62,73.48%,84.39%,62.86%
$646-680,77.00,81.03,66.16%,81.13%,53.53%


## Scores by School Size

* Perform the same operations as above, based on school size.

In [9]:
#Copying the unformatted school summary for use here.
size_df = clean_school_summary.copy()
size_df.drop(["Per Student Budget"], inplace = True, axis = 1)
size_df.drop(["Average Budget"], inplace = True, axis = 1)

#Creating list of school size buckets as group labels
size_list = [0, 1499.9, 2499.9, 3999.9, 5000]
size_bucket_list = ['0 to 1500', '1501-2500', '2501-4000', '4001-5000']

#Bucket List broken down by school size
size_df["School Size Range"] = pd.cut(size_df["Total Students"], 
                                           size_list, labels = size_bucket_list, include_lowest=True)
#Creating a groupby for that set of bins
size_summary_df = size_df.groupby("School Size Range").mean()

size_summary_df.drop(["Total Students"], inplace = True, axis = 1)

#Formatting output
def finalformat(x):
    return "{:.2f}".format(x)

def percentformat(x):
    return "{:.2f}%".format(x)

size_summary_df["Average Math Score"] = size_summary_df["Average Math Score"].apply(finalformat)
size_summary_df["Average Reading Score"] = size_summary_df["Average Reading Score"].apply(finalformat)
size_summary_df["% Passing Math"] = size_summary_df["% Passing Math"].apply(percentformat)
size_summary_df["% Passing Reading"] = size_summary_df["% Passing Reading"].apply(percentformat)
size_summary_df["% Overall Passing"] = size_summary_df["% Overall Passing"].apply(percentformat)

size_summary_df

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Size Range,,,,,
0 to 1500,83.66,83.89,93.50%,96.45%,90.12%
1501-2500,83.36,83.90,93.69%,96.67%,90.62%
2501-4000,76.82,80.96,66.59%,80.39%,53.50%
4001-5000,77.14,80.98,66.50%,81.34%,53.90%


## Scores by School Type

* Perform the same operations as above, based on school type

In [10]:
#Copying the unformatted school summary for use here.
type_df = clean_school_summary.copy()

#Deleting unnecessary columns
type_df.drop(["Per Student Budget"], inplace = True, axis = 1)
type_df.drop(["Average Budget"], inplace = True, axis = 1)
type_df.drop(["Total Students"], inplace = True, axis = 1)

type_summary_df = type_df.groupby("School Type").mean()

type_summary_df["Average Math Score"] = type_summary_df["Average Math Score"].apply(finalformat)
type_summary_df["Average Reading Score"] = type_summary_df["Average Reading Score"].apply(finalformat)
type_summary_df["% Passing Math"] = type_summary_df["% Passing Math"].apply(percentformat)
type_summary_df["% Passing Reading"] = type_summary_df["% Passing Reading"].apply(percentformat)
type_summary_df["% Overall Passing"] = type_summary_df["% Overall Passing"].apply(percentformat)

type_summary_df


,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Type,,,,,
Charter,83.47,83.90,93.62%,96.59%,90.43%
District,76.96,80.97,66.55%,80.80%,53.67%
